In [1]:
import os
import pandas as pd
import numpy as np
import re
from datetime import datetime
import arrow

In [2]:
import numpy as np
import netCDF4 as nc
import pandas as pd
import numpy.ma as ma
import matplotlib as mpl
import matplotlib.pyplot as pltz
import xarray as xar
import rioxarray

In [5]:
# 尝试读取 max 的nc文件
# file_path=r'E:\02data\03typhoon\TY2022_2021010416\TY2022_2021010416\maxSurge_TY2022_2021010416_c0_p00.nc'
# 21-05-04 mac的配置
# file_path=r'/Users/liusihan/data/typhoon_data/TY2022_2021010416/COVERAGE/maxSurge_TY2022_2021010416_c0_p00.nc'
# + 21-07-20 新创建的 TD04 热带风暴的nc
# file_path=r'E:\02data\05docker-data\docker-shared\ty_docker\result\TYTD04_2021071908\maxSurge_TYTD04_2021071908_c0_p00.nc'
# + 21-07-21 P7530
file_path=r'F:\03nginx_data\nmefc_download\TY_GROUP_RESULT\TY2022_2021010416\maxSurge_TY2022_2021010416_c0_p00.nc'

In [6]:
ds_xr=xar.open_dataset(file_path)
ds_xr

<xarray.Dataset>
Dimensions:    (lat: 660, lon: 1080)
Dimensions without coordinates: lat, lon
Data variables:
    latitude   (lat) float32 ...
    longitude  (lon) float32 ...
    max_surge  (lat, lon) float32 ...
Attributes:
    description:  Maximum Storm Surge

In [7]:
ds_xr=ds_xr.rename_vars({'latitude':'y','longitude':'x'})
ds_xr

<xarray.Dataset>
Dimensions:    (lat: 660, lon: 1080)
Dimensions without coordinates: lat, lon
Data variables:
    y          (lat) float32 15.008333 15.025 15.041667 ... 25.975 25.991667
    x          (lon) float32 105.00833 105.025 105.041664 ... 122.975 122.99167
    max_surge  (lat, lon) float32 ...
Attributes:
    description:  Maximum Storm Surge

In [8]:
ds_xr=ds_xr.swap_dims({'lat':'y','lon':'x'})
ds_xr

<xarray.Dataset>
Dimensions:    (x: 1080, y: 660)
Coordinates:
  * y          (y) float32 15.008333 15.025 15.041667 ... 25.975 25.991667
  * x          (x) float32 105.00833 105.025 105.041664 ... 122.975 122.99167
Data variables:
    max_surge  (y, x) float32 ...
Attributes:
    description:  Maximum Storm Surge

In [9]:
ds_xr.rio.set_spatial_dims("x","y",inplace=True)

<xarray.Dataset>
Dimensions:    (x: 1080, y: 660)
Coordinates:
  * y          (y) float32 15.008333 15.025 15.041667 ... 25.975 25.991667
  * x          (x) float32 105.00833 105.025 105.041664 ... 122.975 122.99167
Data variables:
    max_surge  (y, x) float32 ...
Attributes:
    description:  Maximum Storm Surge

In [10]:
ds_xr=ds_xr.rio.write_crs("epsg:4326", inplace=True)
ds_xr

<xarray.Dataset>
Dimensions:      (x: 1080, y: 660)
Coordinates:
  * y            (y) float32 15.008333 15.025 15.041667 ... 25.975 25.991667
  * x            (x) float32 105.00833 105.025 105.041664 ... 122.975 122.99167
    spatial_ref  int32 0
Data variables:
    max_surge    (y, x) float32 ...
Attributes:
    description:   Maximum Storm Surge
    grid_mapping:  spatial_ref

* 注意此处缺少 coords,设置了坐标系

In [8]:
print(ds_xr.info())

xarray.Dataset {
dimensions:
	x = 1080 ;
	y = 660 ;

variables:
	float32 y(y) ;
		y:units = N ;
	float32 x(x) ;
		x:units = E ;
	float32 max_surge(y, x) ;
		max_surge:units = m ;
		max_surge:grid_mapping = spatial_ref ;
	int32 spatial_ref() ;
		spatial_ref:crs_wkt = GEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],CS[ellipsoidal,2],AXIS["geodetic latitude (Lat)",north,ORDER[1],ANGLEUNIT["degree",0.0174532925199433]],AXIS["geodetic longitude (Lon)",east,ORDER[2],ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]] ;
		spatial_ref:semi_major_axis = 6378137.0 ;
		spatial_ref:semi_minor_axis = 6356752.314245179 ;
		spatial_ref:inverse_flattening = 298.257223563 ;
		spatial_ref:reference_ellipsoid_name = WGS 84 ;
		spatial_ref:longitude_of_prime_meridian = 0.0 ;
		spatial_ref:prime_meridian_name = Greenwich ;
		spatial_ref:geographic_crs_name = WGS 84 ;
		spat

ERROR: TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''  
ERROR2: DimensionError: x dimension not found. 'set_spatial_dims()' can address this.

注意使用 rioxarray -> geotiff 时，需要将 lon -> x, lat -> y

注意此处还需要加入一个步骤，对 y 也就是经度倒叙排列

In [9]:
ds_xr['x'][::-1]

<xarray.DataArray 'x' (x: 1080)>
array([122.99167 , 122.975   , 122.958336, ..., 105.041664, 105.025   ,
       105.00833 ], dtype=float32)
Coordinates:
  * x            (x) float32 122.99167 122.975 122.958336 ... 105.025 105.00833
    spatial_ref  int32 0
Attributes:
    units:    E

In [64]:
# ds_xr['x']=ds_xr['x'][::-1]

In [11]:
ds_xr['y']=ds_xr['y'][::-1]

In [36]:
ds_xr

<xarray.Dataset>
Dimensions:      (x: 1080, y: 660)
Coordinates:
  * y            (y) float32 25.991667 25.975 25.958334 ... 15.025 15.008333
  * x            (x) float32 105.00833 105.025 105.041664 ... 122.975 122.99167
    spatial_ref  int32 0
Data variables:
    max_surge    (y, x) float32 ...
Attributes:
    description:   Maximum Storm Surge
    grid_mapping:  spatial_ref

In [13]:
# x dimension not found. 'set_spatial_dims()' can address this.
# '/Users/liusihan/data/typhoon_data/TY2022_2021010416/COVERAGE/maxSurge_TY2022_2021010416_c0_p00.tif'
# output_file=r'E:\02data\05docker-data\docker-shared\ty_docker\result\TYTD04_2021071908\maxSurge_TYTD04_2021071908_c0_p00.tif'
# + 21-07-21 P7530
output_file=r'F:\03nginx_data\nmefc_download\TY_GROUP_RESULT\maxSurge_TY2022_2021010416_c0_p00.tif'

In [14]:
ds_xr.rio.to_raster(output_file)

In [12]:
rio_df=rioxarray.open_rasterio(file_path)
rio_df

C:\Users\evase\.conda\envs\new_oil_gdal\lib\site-packages\rioxarray\_io.py:678: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  warnings.warn(str(rio_warning.message), type(rio_warning.message))


<xarray.DataArray 'max_surge' (band: 1, y: 660, x: 1080)>
[712800 values with dtype=float32]
Coordinates:
  * band         (band) int32 1
  * y            (y) float64 0.5 1.5 2.5 3.5 4.5 ... 656.5 657.5 658.5 659.5
  * x            (x) float64 0.5 1.5 2.5 3.5 ... 1.078e+03 1.078e+03 1.08e+03
    spatial_ref  int32 0
Attributes:
    units:         m
    _FillValue:    999.0
    scale_factor:  1.0
    add_offset:    0.0
    grid_mapping:  spatial_ref

In [ ]:
rio_df.to_